## Data preparation and settings

### Use in Colab to resolve environment (otherwise ignore)

In [ ]:
%%capture
!pip install pytorch-lightning
!pip install transformers
!pip install adapter-transformers
!pip install scikit-learn 

In [ ]:
!nvidia-smi

### Data Inspection

In [ ]:
import json
import pandas as pd
import torch
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset

In [ ]:
# toy dataReader for exploration 
class DataReader:
    def __init__(self, json_name, shuffle=False):
        with open(json_name, 'r') as json_file:
            raw_json = list(json_file)
        self.raw = raw_json
        self.raw_objects = []
        for item in self.raw:
            self.raw_objects.append(json.loads(item))
        self.df = pd.DataFrame(self.raw_objects)


    def get_stats(self):   
        return self.df.head()

    def get_data(self):
        # import IPython; IPython.embed(); exit(1)
        return self.df['text'], self.df['intent']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_path = "./drive/MyDrive/DL project/project/acl-arc/train.jsonl"
val_path = "./drive/MyDrive/DL project/project/acl-arc/dev.jsonl"
test_path = "./drive/MyDrive/DL project/project/acl-arc/test.jsonl"
train_acl = DataReader(json_name=train_path).df
test_acl = DataReader(json_name=val_path).df

In [ ]:
train_acl.head()

In [ ]:
# Simple label view
labels = list(set(train_acl['intent']))
train_acl.groupby('intent').count()['text'].plot.bar()

In [ ]:
attributes = ['Background', 'CompareOrContrast', 'Extends', 'Future', 'Motivation', 'Uses']

### Dataset

In [ ]:
class ACL_Dataset(Dataset):
    def __init__(self, data_path, tokenizer, attributes, max_token_len=128):
        self.data_path = data_path
        self.tokenizer = tokenizer
        self.attribute = attributes
        self.max_token_len = max_token_len
        self._prepare_data()

    def _prepare_data(self):
        '''
        Place to add other data preparations (sampling / train&test separation)

        '''
        if self.data_path[-3:] != "csv":
            with open(self.data_path, 'r') as json_file:
                raw_json = list(json_file)
            raw_objects = []
            for item in raw_json:
                raw_objects.append(json.loads(item))
            self.data = pd.DataFrame(raw_objects)

            # Turn into one-hot encoding
            encoder = OneHotEncoder(handle_unknown='ignore')
            encoder_df = pd.DataFrame(encoder.fit_transform(self.data[['intent']]).toarray())
            encoder_df.columns = self.attribute
            self.data = self.data.join(encoder_df)

        else:
            print("Not yet implemented for csv")

    def __len__ (self):
        return len(self.data)

    def __getitem__(self, index):
        '''
            samples and labels loaded here 
        '''
        item = self.data.iloc[index]
        labels = torch.Tensor(item[self.attribute])
        text = str(item.cleaned_cite_text)
        tokens = self.tokenizer.encode_plus(text, add_special_tokens=True, 
                    return_tensors='pt', truncation=True, max_length = self.max_token_len, 
                    padding="max_length", return_attention_mask=True)
    
        return {"input_ids": tokens.input_ids.flatten(), "attention_mask": tokens.attention_mask.flatten(), "labels": labels}

In [ ]:
from transformers import RobertaTokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
train = ACL_Dataset(train_path, tokenizer, attributes=attributes)


In [ ]:
train.data[['intent', 'CompareOrContrast', 'Uses', 'Background', 'Future', 'Motivation', 'Extends']]

### Data module

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

In [ ]:
class ACL_DataLoader(pl.LightningDataModule):
    def __init__(self, train_path, val_path, test_path, attributes, batch_size:int = 32, max_token_length: int = 128, model_name = "roberta-base"):
        super().__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.batch_size = batch_size
        self.max_token_length = max_token_length
        self.model_name = model_name
        self.attributes = attributes
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)


    def setup(self, stage=None):
        if stage in (None, "fit"):
            self.train_dataset = ACL_Dataset(train_path, self.tokenizer, attributes=self.attributes)
            self.val_dataset = ACL_Dataset(val_path, self.tokenizer, attributes=self.attributes)

        if stage == "predict":
            self.val_dataset = ACL_Dataset(val_path, self.tokenizer, attributes=self.attributes)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers = 4, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers = 4, shuffle=False)

    def predict_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers = 4, shuffle=False)

In [ ]:
acl_datamodule = ACL_DataLoader(train_path=train_path, val_path=val_path, test_path=test_path, attributes=attributes)
acl_datamodule.setup()
acl_dataloader = acl_datamodule.train_dataloader()

### Model

In [ ]:
attributes

In [ ]:
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
from transformers import RobertaForSequenceClassification, RobertaModelWithHeads
from torchmetrics.functional import f1_score, accuracy
import torch.nn as nn
import math

'''
TODO change model accordingly
'''
class ACL_Classifier(pl.LightningModule):

    def __init__(self, config: dict, adapter_config: dict):
        super().__init__()
        self.config = config

        # Static classification head is used
        self.pre_trained_model = RobertaForSequenceClassification.from_pretrained(config['model_name'], 
                                                                                  problem_type="multi_label_classification", 
                                                                                  num_labels=self.config['n_labels'],
                                                                                  return_dict = True)
        
        # May switch to dynamic ones 
        # self.pre_trained_model = RobertaModelWithHeads.from_pretrained(config = config)
        # self.pre_trained_model.add_adapter("citation-intent", config=adapter_config)

        # # TODO change the parameters
        # # model.add_classification_head(
        # #     "rotten_tomatoes",
        # #     num_labels=2,
        # #     id2label={ 0: "👎", 1: "👍"}
        # # )

        # # Enable adapter training 
        # self.pre_trained_model.train_adapter("citation-intent") 
        # self.pre_trained_model.set_active_adapters(["citation-intent"])   
        self.softmax = nn.Softmax()


    def forward(self, input_ids, attention_mask, labels):
        # pre_trained model output
        logits = self.pre_trained_model(input_ids=input_ids, attention_mask=attention_mask).logits
        loss = self.pre_trained_model(input_ids=input_ids, labels=labels).loss
        output = self.softmax(logits)
        return loss, output
    
    def training_step(self, batch, batch_index):
        loss, output = self.forward(**batch)
        target = torch.argmax(batch["labels"], axis=1)
        acc = accuracy(output, target)
        self.log("train_loss", loss)
        self.log("train_acc_step", acc)
        return {"loss": loss, "accuracy": acc, "predictions": output, "labels": batch["labels"]}

    
    def validation_step(self, batch, batch_index):
        loss, output = self.forward(**batch)
        target = torch.argmax(batch["labels"], axis=1)
        f1 = f1_score(output, target)
        self.log("val_loss", loss)
        self.log("val_f1_score", f1)
        return {"val_loss": loss, "f1_score": f1, "predictions": output, "labels": batch["labels"]}

    def test_step(self, batch, batch_index):
        _, output = self.forward(**batch)
        target = torch.argmax(batch["labels"], axis=1)
        f1 = f1_score(output, target)
        self.log("val_f1_score", f1) 
        return output
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config['lr'], weight_decay=self.config['weight_decay'] ,no_deprecation_warning=True, correct_bias=False)
        total_steps = self.config['train_size'] / self.config['batch_size']
        warmup_steps = math.floor(total_steps * self.config['warmup'])
        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        return [optimizer], [scheduler]

In [ ]:
from transformers import HoulsbyConfig

adapter_config = HoulsbyConfig()

roberta_config = {
    # Some randomly typed-in initial configs
    'model_name': 'roberta-base',
    'batch_size': 16,
    'lr': 2e-5,
    'warmup': 0.06,
    'weight_decay': 0.01,
    'n_epochs': 10,
    'train_size': len(acl_datamodule.train_dataloader()),
    'n_labels': len(labels)

}

adapter_config


In [ ]:
'''Single output sanity check'''
model = ACL_Classifier(config=roberta_config, adapter_config=adapter_config)
idx = 0
input_ids = train.__getitem__(idx)['input_ids']
attention_mask = train.__getitem__(idx)['attention_mask']
labels = train.__getitem__(idx)['labels']
loss, output = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0), labels.unsqueeze(0))
print("loss" + str(loss))
print("raw prediction: " + str(output))
print("label: " + str(labels))

In [ ]:
# Copy datamodule here - for convenience
acl_datamodule = ACL_DataLoader(train_path=train_path, val_path=val_path, test_path=test_path, attributes=attributes, batch_size=roberta_config['batch_size'])
acl_datamodule.setup()
model = ACL_Classifier(config=roberta_config, adapter_config=adapter_config)

# Auto-Training loop
trainer = pl.Trainer(max_epochs=roberta_config['n_epochs'], gpus=1, num_sanity_val_steps=1, default_root_dir='/content/drive/MyDrive')
trainer.fit(model, acl_datamodule)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./drive/MyDrive/lightning_logs

### Evaluation

In [ ]:
# saved_model = ACL_Classifier.load_from_checkpoint('./drive/MyDrive/lightning_logs/version_4/checkpoints/saved.ckpt', config=roberta_config, adapter_config=adapter_config)

In [ ]:
import numpy as np
test_dataloaders = acl_datamodule.predict_dataloader()
trainer.test(model ,dataloaders=test_dataloaders)

